# Practical Week 11: Sequence-to-sequence models

There are three important tasks to complete this week:

1. Learning how to use LSTM and Transformers to process data sequences. You will find yourself waiting a long time
   for many of these computations to complete. While you wait:

2. Complete the course survey. It's in ilearn, and this link will show you how to view it and complete it:
https://ishare.mq.edu.au/prod/items/912f85f6-70e8-48a5-94ca-d3928e9d7c3c/1/viewcontent/


3. Sign up to get API access to OpenAI, which you will need for next week's practical. https://platform.openai.com/signup 



## Sequence to sequence

We're going to do a task that even ChatGPT-4 doesn't get right: arithmetic on long integer sequences.

The following cell creates a list of example bitwise-or calculations using python's `|` operator.

If you aren't familiar with Python's bit-handling functions, or what a bitwise-or does, here's a short
introduction: https://realpython.com/python-bitwise-operators/#bitwise-logical-operators

In [157]:
math_facts = []
for i in range(64):
    for j in range(64):
        k = i | j
        math_facts.append(f"{i:06b} | {j:06b} = {k:06b}.")

Import the random library (or any other way you like) to see a few samples from the `math_facts` list.

In [138]:
import random
random.sample(math_facts,5)

['001100 | 000010 = 001110.',
 '011110 | 001110 = 011110.',
 '100000 | 010000 = 110000.',
 '101000 | 000100 = 101100.',
 '100110 | 101110 = 101110.']

### Data Preparation

We need to process these strings so that they can be used in a sequence-to-sequence model.

- For each string, split it up on the equals sign.
- On the left hand side of the equals sign, add a space between each digit so that each digit is its own token
- On the right hand side of the equals sign, generate 9 training samples. Encode 0 as `tf.convert_to_tensor([1,0,0])`, 1 as `tf.convert_to_tensor([0,1,0])` and "." as `tf.convert_to_tensor([0,0,1])`

e.g. suppose you were working with the statement:

```001000 | 010101 = 011101.```

Then you want to turn this into:

```
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 =', 'next_token': '0'}, 
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0', 'next_token': '1'}, 
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1', 'next_token': '1'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1', 'next_token': '1'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1 1', 'next_token': '0'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1 1 0', 'next_token': '1'},
{'context': '0 0 1 0 0 0 | 0 1 0 1 0 1 = 0 1 1 1 0 1', 'next_token': '.'},
```

In [129]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf

r = re.compile('(\d)(\d)(\d)(\d)(\d)(\d)'
               ' \| '
               '(\d)(\d)(\d)(\d)(\d)(\d)'
               ' = '
               '(\d)(\d)(\d)(\d)(\d)(\d)\.'
              )
data = []
token_lookup = {'0': [1,0,0], '1': [0,1,0]}

for fact in math_facts:
    g = r.search(fact)
    lhs = f"{g[1]} {g[2]} {g[3]} {g[4]} {g[5]} {g[6]} | "
    lhs += f"{g[7]} {g[8]} {g[9]} {g[10]} {g[11]} {g[12]} ="
    
    data.append({'context': lhs, 'next_token': token_lookup[g[13]]} )
    lhs += f" {g[13]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[14]]} )
    lhs += f" {g[14]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[15]]} )
    lhs += f" {g[15]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[16]]} )
    lhs += f" {g[16]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[17]]} )
    lhs += f" {g[17]}"
    data.append({'context': lhs, 'next_token': token_lookup[g[18]]} )
    lhs += f" {g[18]}"
    #data.append({'context': lhs, 'next_token': tf.convert_to_tensor([0,0,1])} )
    data.append({'context': lhs, 'next_token': [0,0,1]} )

df = pd.DataFrame.from_records(data)

View a sample of the `df` dataframe to confirm it matches what you expect.

In [130]:
df.sample(5)

,context,next_token
6835,0 0 1 1 1 1 ^ 0 1 0 0 0 0 = 0 1 1,"[0, 1, 0]"
27678,1 1 1 1 0 1 ^ 1 1 0 0 1 0 =,"[0, 1, 0]"
18078,1 0 1 0 0 0 ^ 0 1 0 1 1 0 = 1 1 1 1,"[0, 1, 0]"
14047,0 1 1 1 1 1 ^ 0 1 0 1 1 0 = 0 1 1 1 1,"[0, 1, 0]"
10958,0 1 1 0 0 0 ^ 0 1 1 1 0 1 = 0 1 1,"[0, 1, 0]"


Split the `df` dataframe up into train, validation and test data.

In [131]:
import sklearn.model_selection

trainval, test = sklearn.model_selection.train_test_split(df)
train, validation = sklearn.model_selection.train_test_split(trainval)

Create a `keras.layers.TextVectorization` object with the following constructor parameters:

- split="whitespace"
- standardize=None
- output_sequence_length=26 (the longest possible sequence)

Adapt it to the training data (the `context` column) and vectorise the training, validation and test texts.

(Bonus challenge: this is one of the few times where you can actually give the vectorizer a vocabulary, and
you don't have to adapt from training data. Can you figure out how?)

In [132]:
import keras
vectoriser = keras.layers.TextVectorization(
    split="whitespace", standardize=None, 
    output_sequence_length=21
)
vectoriser.adapt(train.context)

train_data = vectoriser(train.context)
validation_data = vectoriser(validation.context)
test_data = vectoriser(test.context)

2023-05-14 15:53:27.096359: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


The target values need to be converted into tensorflow Tensors. Unfortunately, `tf.convert_to_tensor` doesn't
handle pandas Series correctly. Use this work-around:

`train_y = tf.convert_to_tensor(list(train.next_token))`

(Create `train_y`, `validation_y` and `test_y`).

In [133]:
train_y = tf.convert_to_tensor(list(train.next_token))
validation_y = tf.convert_to_tensor(list(validation.next_token))
test_y = tf.convert_to_tensor(list(test.next_token))

Create a model with:

- an input layer

- an embedding layer (a 4-dimensional embedding space will be sufficient)

- a bidirectional LSTM layer with 16 neurons in it

- an output layer of 3 dense neurons with a softmax activation

Compile it, and use a loss function of `categorical_crossentropy`

In [134]:
inputs = keras.Input(shape=(21,))
embedding = keras.layers.Embedding(
    input_dim=4, output_dim=4)(inputs)
thinking_layer = keras.layers.Bidirectional(
    keras.layers.LSTM(16))(embedding)
output_layer = keras.layers.Dense(
    3,
    activation="softmax")(thinking_layer)
model = keras.Model(
    inputs=inputs,
    outputs=output_layer)
model.summary()
model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=['accuracy'])

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 21)]              0         
                                                                 
 embedding_8 (Embedding)     (None, 21, 4)             16        
                                                                 
 bidirectional_8 (Bidirectio  (None, 32)               2688      
 nal)                                                            
                                                                 
 dense_8 (Dense)             (None, 3)                 99        
                                                                 
Total params: 2,803
Trainable params: 2,803
Non-trainable params: 0
_________________________________________________________________


Fit the model using the training data, add validation data and early stopping. You'll find that it 
reaches 100% accuracy after about 100-120 epochs.

Surprisingly, it doesn't ever overfit. Make a graph of it if you want to (or just watch the numbers
fly past as it trains).

While you are waiting for this to complete, it would be a great time to **do the student evaluation survey now**.

In [135]:
callbacks = [
    keras.callbacks.EarlyStopping(patience=10, monitor="val_accuracy")
]
history = model.fit(x=train_data, y=train_y,
          validation_data=(validation_data, validation_y),
          epochs=200,
                    callbacks=callbacks
)

Epoch 1/200


2023-05-14 15:53:32.503717: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-14 15:53:32.698062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-14 15:53:32.708601: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-14 15:53:32.868202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-14 15:53:32.882677: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


504/504 [==============================] - ETA: 0s - loss: 0.5933 - accuracy: 0.7501

2023-05-14 15:53:45.549598: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-14 15:53:45.624862: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-05-14 15:53:45.631813: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


504/504 [==============================] - 16s 29ms/step - loss: 0.5933 - accuracy: 0.7501 - val_loss: 0.4517 - val_accuracy: 0.7900
Epoch 2/200
504/504 [==============================] - 14s 28ms/step - loss: 0.4515 - accuracy: 0.7889 - val_loss: 0.4462 - val_accuracy: 0.7919
Epoch 3/200
504/504 [==============================] - 14s 28ms/step - loss: 0.4517 - accuracy: 0.7883 - val_loss: 0.4390 - val_accuracy: 0.7954
Epoch 4/200
504/504 [==============================] - 14s 28ms/step - loss: 0.4431 - accuracy: 0.7941 - val_loss: 0.4519 - val_accuracy: 0.7963
Epoch 5/200
504/504 [==============================] - 14s 28ms/step - loss: 0.4324 - accuracy: 0.7969 - val_loss: 0.4351 - val_accuracy: 0.7995
Epoch 6/200
504/504 [==============================] - 14s 28ms/step - loss: 0.4314 - accuracy: 0.8024 - val_loss: 0.4410 - val_accuracy: 0.8049
Epoch 7/200
504/504 [==============================] - 14s 28ms/step - loss: 0.4168 - accuracy: 0.8060 - val_loss: 0.4374 - val_accuracy: 0.79

504/504 [==============================] - 14s 28ms/step - loss: 5.1614e-04 - accuracy: 0.9999 - val_loss: 1.4389e-04 - val_accuracy: 0.9998
Epoch 113/200
504/504 [==============================] - 14s 29ms/step - loss: 0.0012 - accuracy: 0.9998 - val_loss: 6.1805e-06 - val_accuracy: 1.0000
Epoch 114/200
504/504 [==============================] - 15s 29ms/step - loss: 6.6695e-05 - accuracy: 1.0000 - val_loss: 3.2481e-05 - val_accuracy: 1.0000
Epoch 115/200
504/504 [==============================] - 14s 28ms/step - loss: 0.0026 - accuracy: 0.9998 - val_loss: 7.9285e-06 - val_accuracy: 1.0000
Epoch 116/200
504/504 [==============================] - 14s 29ms/step - loss: 6.9430e-04 - accuracy: 0.9998 - val_loss: 1.0025e-06 - val_accuracy: 1.0000
Epoch 117/200
504/504 [==============================] - 15s 30ms/step - loss: 7.7533e-04 - accuracy: 0.9997 - val_loss: 1.2049e-05 - val_accuracy: 1.0000
Epoch 118/200
504/504 [==============================] - 15s 29ms/step - loss: 0.0015 - accu

## Student evaluation survey

Set this value to true when you have done your student evaluation survey.

In [ ]:
I_have_completed_my_student_evaluation_survey = False

## OpenAI API key

Set this value to true when you have signed up for your API key.

Remember that the API key can only be viewed once, so save it somewhere safe

In [ ]:
I_have_signed_up_for_openai = False

------------

## Using the sequence-to-sequence model

Check the test accuracy. It should be very good!

In [158]:
model.evaluate(train_data, train_y)

504/504 [==============================] - 7s 14ms/step - loss: 5.1740e-10 - accuracy: 1.0000


[5.174013439912528e-10, 1.0]

Create a function that takes a bitwise-or arithmetic problem, an optional list of already-returned tokens and returns the next token.

e.g. 
`get_next_token('1 0 0 1 0 0 | 0 0 0 0 0 1', '1 0 0')` should return "1".

You will need to:

- concatentate the first and second arguments (with an equals sign (=) between them

- vectorize that input string

- use the model to predict the probabilities of the three options for the next token (0,1 or .)

- find the option that has the highest probability

- return that as a token

In [149]:
def get_next_token(arithmetic_problem, part_decoded_already=""):
    text = arithmetic_problem + " = " + part_decoded_already
    text_vector = vectoriser([text])
    probabilities = model.predict(text_vector, verbose=0)[0]
    index_of_best = np.argmax(probabilities)
    if index_of_best == 2:
        return '.'
    else:
        return str(index_of_best)

Test it out with a sample problem.

In [150]:
get_next_token('1 0 0 1 0 0 | 0 0 0 0 0 1', '1 0 0 1 0 1')

'.'

Now create a function `get_full_answer` that calls `get_next_token` repeatedly with more context each time
until it gets a `.`. 

In [155]:
def get_full_answer(arithmetic_problem):
    part_decoded_already = ""
    while True:
        n = get_next_token(arithmetic_problem, part_decoded_already)
        if n.strip() == '.':
            break
        part_decoded_already += ' ' + n
    return part_decoded_already


Try it out with a sample problem

In [156]:
get_full_answer('1 0 0 1 0 0 | 0 0 0 0 0 1')

' 1 0 0 1 0 1'

# Transformers (Optional)

Depending on how fast your computer is, how long it takes to do the student survey and so on, you 
might or might not have a lot of time left over.

If you have run through all the above very quickly, go through the
https://www.tensorflow.org/text/tutorials/transformer

The tutorial is slightly out-of-date (as almost everything involving transformers is!)
because it re-implements components that already exist in the `keras_nlp` library, but it
is otherwise excellent.